# NYC Parking Tickets EDA 

In [1]:
%%local
import plotly
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from plotly.graph_objs import *

In [3]:
%%configure -f
{
"conf": {
    "spark.pyspark.python": "python3", 
    "spark.pyspark.virtualenv.enabled": "true", 
    "spark.pyspark.virtualenv.type": "native", 
    "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv",
    "spark.app.name":"694"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
51,application_1638573803091_0052,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
51,application_1638573803091_0052,pyspark,idle,Link,Link,✔


In [4]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=694>

In [5]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
51,application_1638573803091_0052,pyspark,idle,Link,Link,✔


In [6]:
# package installation
# install packages plotly
# sc.install_pypi_package("plotly")
# sc.list_packages()
# sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# EDA 

## Read in Data

In [7]:
#Columns we use
# Plate ID 哪些车违章的次数多 #2                    
# Registration State 可以看这些车是哪里的车牌 #3     
# Issue Date #5   
# Violation Code #6
# Vehicle Body Type #7                           
# Vehicle Make #8
# Violation Time #20
# Violation County #22
# Street Name #25
# From Hours In Effect #32
# To Hours In Effect #33
# Vehicle color #34
# Vehicle Year #36
# Vehicle Expiration Date #13

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
## load data in S3
data_path1 = "s3://msds694-project-renne/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv"
data1 = sc.textFile(data_path1)
data_path2 = "s3://msds694-project-renne/Parking_Violations_Issued_-_Fiscal_Year_2015.csv"
data2 = sc.textFile(data_path2)
data = data1.union(data2)
data = data.map(lambda x: x.split(',')).map(lambda x: (x[1:3]+[x[4]]+x[6:8]+[x[19]]+[x[24]]+x[31:34]+[x[35]]+[x[21]]+[x[12]]+[x[5]]))
# data.take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# sample
# rdd = data.sample(False,0.005,seed=1)
rdd = data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#put this at the begining
# rdd.unpersist()
 #put this at the end we’d like to test/ in the end


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Without cache

In [11]:
rdd.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[5] at RDD at PythonRDD.scala:53

In [12]:
import time
#without cache
start = time.time()
def trans(data):
    try:
        if 'A' in data:
            data = data.replace('A','')
            data = data[:2]+':'+ '00'
        elif 'P' in data:
            data = data.replace('P','')
            d = int(data[1:2])+12
            data = str(d)+':'+ data[2:]
            data = data[:2]+':'+'00'
    except:
        data = 'K'
    return data
Q = rdd.map(lambda x: (x[5],x[1])).map(lambda x: (trans(x[0]),x[1])).countByKey()
Q = sorted(Q.items(),key=lambda x: x[0],reverse=False)
Q_rdd = sc.parallelize(Q)
Q_rdd_df = Q_rdd.filter(lambda x: x[1] > 21).filter(lambda x: x[0] != '').toDF(['Issue_Time','Ticket_Count'])
Q_rdd_df.createOrReplaceTempView("Q_rdd_view")
Q_rdd_df.show()
print("second", time.time() - start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+
|Issue_Time|Ticket_Count|
+----------+------------+
|     00:00|      117255|
|     01:00|      207589|
|     02:00|      180240|
|     03:00|      140854|
|     04:00|       64682|
|     05:00|      159085|
|     06:00|      422918|
|     07:00|     1030195|
|     08:00|     1886260|
|     09:00|     2176025|
|     10:00|     1743116|
|     11:00|     2117534|
|     12:00|      316201|
|     13:00|     2182580|
|     14:00|     3492323|
|     15:00|     1276283|
|     16:00|     1216548|
|     17:00|      881265|
|     18:00|      561913|
|     19:00|      168937|
+----------+------------+
only showing top 20 rows

second 69.39213395118713

In [13]:
rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

71

## What's the Number of car stickers for each time period (count)?_Zhiyi Ren

In [14]:
import time
#with cache
start = time.time()
def trans(data):
    try:
        if 'A' in data:
            data = data.replace('A','')
            data = data[:2]+':'+ '00'
        elif 'P' in data:
            data = data.replace('P','')
            d = int(data[1:2])+12
            data = str(d)+':'+ data[2:]
            data = data[:2]+':'+'00'
    except:
        data = 'K'
    return data
Q = rdd.map(lambda x: (x[5],x[1])).map(lambda x: (trans(x[0]),x[1])).countByKey()
Q = sorted(Q.items(),key=lambda x: x[0],reverse=False)
Q_rdd = sc.parallelize(Q)
Q_rdd_df = Q_rdd.filter(lambda x: x[1] > 21).filter(lambda x: x[0] != '').toDF(['Issue_Time','Ticket_Count'])
Q_rdd_df.createOrReplaceTempView("Q_rdd_view")
Q_rdd_df.show()
print("second", time.time() - start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+
|Issue_Time|Ticket_Count|
+----------+------------+
|     00:00|      117255|
|     01:00|      207589|
|     02:00|      180240|
|     03:00|      140854|
|     04:00|       64682|
|     05:00|      159085|
|     06:00|      422918|
|     07:00|     1030195|
|     08:00|     1886260|
|     09:00|     2176025|
|     10:00|     1743116|
|     11:00|     2117534|
|     12:00|      316201|
|     13:00|     2182580|
|     14:00|     3492323|
|     15:00|     1276283|
|     16:00|     1216548|
|     17:00|      881265|
|     18:00|      561913|
|     19:00|      168937|
+----------+------------+
only showing top 20 rows

second 16.188396215438843

In [15]:
%%sql -o Q_rdd_table
SELECT *
FROM Q_rdd_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## What's the Top10 Reasons for getting tickets? _Zhiyi Ren

In [16]:
# Violation code
def violationinfo(x):
    if x == '21':
        x = x.replace('21','No parking: Street cleaning')
    elif x == '38':
        x = x.replace('38','Failing to show a Parking Meter receipt')
    elif x == '14':
        x = x.replace('14','No Standing: Day/Time limits')
    elif x == '37':
        x = x.replace('37','Parking expire Parking Meter allowed time')
    elif x == '71':
        x = x.replace('71','Inspection sticker w/o or missing')
    elif x == '7':
        x = x.replace('7','Fail to stop at red light')
    elif x == '20':
        x = x.replace('20','No Parking: Day/Time limits')
    elif x == '46':
        x = x.replace('46','Double Parking')
    elif x == '36':
        x = x.replace('36','Speed violation at/near school zone')
    elif x == '40':
        x = x.replace('40','Parking closer than a fire hydrant')
    return x

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
Q = rdd.map(lambda x: ((x[-1],x[1]),1))
Q = Q.reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1],ascending=False)
Q = Q.map(lambda x: (x[0][0],x[0][1],x[1])).filter(lambda x: x[1]=='NY')
violation_code = sc.parallelize(Q.take(10))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
violation_code = violation_code.map(lambda x: (violationinfo(x[0]),x[1],x[2]))
violation_code_df = violation_code.toDF(['violation code','county','ticket count'])
violation_code_df.createOrReplaceTempView("violation_code_view")
violation_code_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+------------+
|      violation code|county|ticket count|
+--------------------+------+------------+
|No parking: Stree...|    NY|     2142501|
|Failing to show a...|    NY|     2094690|
|No Standing: Day/...|    NY|     1281866|
|Parking expire Pa...|    NY|     1237768|
|Inspection sticke...|    NY|     1091417|
|Fail to stop at r...|    NY|      954114|
|No Parking: Day/T...|    NY|      911469|
|      Double Parking|    NY|      829914|
|Speed violation a...|    NY|      809167|
|Parking closer th...|    NY|      699951|
+--------------------+------+------------+

In [19]:
%%sql -o violation_code_table
SELECT * FROM violation_code_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
%%local
x = violation_code_table['violation code']
y = violation_code_table['ticket count']
fig = go.Figure(data=[go.Pie(labels=x,values=y,pull=[0.1, 0.1, 0, 0])],layout=go.Layout(title=go.layout.Title(text="Top10 Violation Reason in NY County")))
fig.update_layout(xaxis=dict(tickformat='digit'))

fig.show()

## What's the top10 brand was been issued Parking Tickets? _Zhiyi Ren

In [21]:
Q1 = rdd.map(lambda x: (x[4],x[1])).countByKey()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
brand_count_top10 = sorted(Q1.items(),key=lambda x: x[1],reverse=True)[:10]
brand_count_top10 = sc.parallelize(brand_count_top10)
brand_count_top10_df = brand_count_top10.toDF(['Brand','Ticket Count'])
brand_count_top10_df.createOrReplaceTempView("brand_count_top10_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
%%sql -o brand_top_10_table
SELECT * FROM brand_count_top10_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [24]:
%%local
layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)
fig = go.Figure(layout=layout)
x = brand_top_10_table['Brand']
y = brand_top_10_table['Ticket Count']
fig = go.Figure(data=go.Bar(x=x,y=y),layout=go.Layout(title=go.layout.Title(text="Top10 Brands that Get Tickets"),    
                                                      paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'))
fig.update_layout(xaxis=dict(tickformat='digit'))
fig.show()

## What's the top7 car's color was been issued Parking Tickets?_Roger Ren

In [25]:
def colorcombine(data):
    for x in data:
        if data == 'GY':
            data=data.replace('GY','GREY')
        if data == 'WH':
            data=data.replace('WH','WHITE')
        if data == 'BK':
            data=data.replace('BK','BLACK')
        if data == 'BL':
            data = data.replace('BL','BLUE')
        if data == 'RD':
            data = data.replace('RD','RED')
    return data
Q2 = rdd.map(lambda x: (x[9],x[1])).map(lambda x: (colorcombine(x[0]),x[1])).countByKey()
color_count_top7 = sorted(Q2.items(),key=lambda x: x[1],reverse=True)[:7]
color_count_top7 = sc.parallelize(color_count_top7)
color_count_top7_df = color_count_top7.toDF(['Color','Ticket Count'])
color_count_top7_df.createOrReplaceTempView("color_count_top7_view")
color_count_top7_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------+
|Color|Ticket Count|
+-----+------------+
|WHITE|     5812505|
| GREY|     3818991|
|BLACK|     3743621|
| BLUE|     1684221|
|  RED|     1221939|
|SILVE|      672367|
|BROWN|      626928|
+-----+------------+

In [26]:
%%sql -o color_top7_table
SELECT * FROM color_count_top7_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [27]:
%%local
x = color_top7_table['Color']
y = color_top7_table['Ticket Count']
fig = go.Figure(data=[go.Pie(labels=x,values=y)],layout=go.Layout(title=go.layout.Title(text="Top7 Car Colors that Get Tickets")))
fig.update_layout(xaxis=dict(tickformat='digit'))
fig.update_traces(marker=dict(colors=['ghostwhite', 'grey', 'black','blue','red','silver','saddlebrown']))
fig.show()

## Which car was ticketed during this period_Zhiyi Ren

In [28]:
Q3 = rdd.map(lambda x: (x[0],x[1])).countByKey()
car_ticketcount_top10 = sorted(Q3.items(),key=lambda x: x[1],reverse=True)[:12]
car_ticketcount_top10 = sc.parallelize(car_ticketcount_top10).filter(lambda x: x[0]!= 'N/A').filter(lambda x: x[0]!= 'BLANKPLATE')
car_ticketcount_top10_df = car_ticketcount_top10.toDF(['Plate ID','Ticket Count'])
car_ticketcount_top10_df.createOrReplaceTempView("car_ticketcount_top10_view")
car_ticketcount_top10_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+
|Plate ID|Ticket Count|
+--------+------------+
| 49839JG|        2681|
| 92979JE|        2387|
| 44491JW|        2322|
| 49781MA|        2148|
| 2208638|        2142|
| 47603MD|        2107|
|  AN917T|        2015|
| 75225JW|        1960|
| PRZ2525|        1937|
| 63485JM|        1906|
+--------+------------+

In [29]:
%%sql -o car_ticketcount_top10_table
SELECT * FROM car_ticketcount_top10_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [30]:
%%local
x = car_ticketcount_top10_table['Plate ID']
y = car_ticketcount_top10_table['Ticket Count']
fig = go.Figure(data=go.Bar(x=x,y=y),layout=go.Layout(title=go.layout.Title(text="Top10 Car Plate ID that Get Tickets"),    
                                                      paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'))
fig.update_layout(xaxis=dict(tickformat='digit'))
fig.show()

## The Top20 Ave/Street parking tickets being issued_Zhiyi Ren

In [31]:
Q4 = rdd.map(lambda x: ((x[11]+' '+x[6]),1))
Q4 = Q4.reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1],ascending=False)
Top20_street_count = sc.parallelize(Q4.take(20))
Top20_street_count_df = Top20_street_count.toDF(['Street','Ticket Count'])
Top20_street_count_df.createOrReplaceTempView("Top20_street_count_view")
Top20_street_count_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+
|              Street|Ticket Count|
+--------------------+------------+
|         NY Broadway|      333613|
|          NY 3rd Ave|      259345|
|      NY Madison Ave|      196487|
|    NY Lexington Ave|      169317|
|          NY 2nd Ave|      152598|
|          NY 1st Ave|      151529|
|       Q Queens Blvd|      131173|
|    NY Amsterdam Ave|      120289|
|          NY 5th Ave|      109150|
|          NY 8th Ave|      107327|
|           K 5th Ave|      107123|
|       Q Jamaica Ave|      104717|
|          NY 6th Ave|       99601|
|     NY Columbus Ave|       90503|
|          NY 7th Ave|       89088|
|          Q 37th Ave|       82235|
| HORACE HARDING E...|       79656|
|  K Coney Island Ave|       78280|
|         NY Park Ave|       76708|
|     Q Roosevelt Ave|       74486|
+--------------------+------------+

In [32]:
%%sql -o Top20_street_count_table
SELECT * FROM Top20_street_count_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [33]:
%local
fig = go.Figure(go.Bar(
            x=Top20_street_count_table['Ticket Count'],
            y=Top20_street_count_table['Street'],
            orientation='h'),layout=go.Layout(title=go.layout.Title(text="Top20 Streets with Car Tickets"),    
                                                      paper_bgcolor='rgba(0,0,0,0)'))

fig.show()

## Issue Date Timeline_Zhiyi Ren

In [34]:
def skip(data):
    try:
        data = int(data)
    except:
        data = 0
    return data
        
Q5 = rdd.map(lambda x: (x[2],(x[1]))).filter(lambda x: x[0]!="").map(lambda x: (((x[0][-4:]+'/'+x[0][:2]),x[1])))
Q5 = Q5.filter(lambda x: skip(x[0][:4]) >= 2013)
Q5 = Q5.filter(lambda x: skip(x[0][:4]) <= 2015).sortByKey().countByKey()

issue_ticket_count = sorted((Q5.items()),key=lambda x: x[0])
issue_ticket_count = sc.parallelize(issue_ticket_count)
issue_ticket_count_df = issue_ticket_count.toDF(['Date','Ticket Count'])
issue_ticket_count_df.sort("Date",'Ticket Count')

issue_ticket_count_df.createOrReplaceTempView("issue_ticket_count_view")
issue_ticket_count_df.show()
issue_ticket_count_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
|   Date|Ticket Count|
+-------+------------+
|2013/01|        1619|
|2013/02|         390|
|2013/03|         485|
|2013/04|         585|
|2013/05|         286|
|2013/06|        3359|
|2013/07|      172799|
|2013/08|      838361|
|2013/09|      826787|
|2013/10|      952878|
|2013/11|      848071|
|2013/12|      734045|
|2014/01|      722616|
|2014/02|      654134|
|2014/03|      913907|
|2014/04|      892570|
|2014/05|      952629|
|2014/06|      950036|
|2014/07|      969958|
|2014/08|      911815|
+-------+------------+
only showing top 20 rows

root
 |-- Date: string (nullable = true)
 |-- Ticket Count: long (nullable = true)

In [35]:
%%sql -o issue_ticket_count_table
SELECT * 
FROM issue_ticket_count_view
WHERE issue_ticket_count_view.Date >= '2013/08'
and issue_ticket_count_view.Date <= '2015/06'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [36]:
%local
title = 'The Number of Car Tickers Issued by Time'
labels = ['Ticket Count']
colors = ['rgb(67,67,67)']
# 'rgb(115,115,115)', 'rgb(49,130,189)', 'rgb(189,189,189)'
mode_size = [8, 8, 12, 8]
line_size = [2, 2, 4, 2]

x_data = [issue_ticket_count_table['Date']]
y_data = np.array([issue_ticket_count_table['Ticket Count']])

layout = Layout(
    paper_bgcolor='rgba(1,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)
fig = go.Figure(layout=layout)

for i in range(0, 1):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color='darkgreen', width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x= [x_data[i][17]],
        y= [y_data[i][17]],
        mode='markers',
        marker=dict(color='black', size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(67,67,67)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        linecolor='rgb(67,67,67)',
        ticks='outside',
        zeroline=True,
        showline=True,
        linewidth=2,
        showticklabels=True,
    ),
    
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
# for y_trace, label, color in zip(y_data, labels, colors):
#     # labeling the left_side of the plot
#     annotations.append(dict(xref='paper', x=0.05, y=y_trace[-1],
#                                   xanchor='right', yanchor='middle',
#                                   text=label.format(y_trace[0]),
#                                   font=dict(family='Arial',
#                                             size=16),
#                                   showarrow=False))
    # labeling the right_side of the plot
#     annotations.append(dict(xref='paper', x=0.95, y=y_trace[5],
#                                   xanchor='left', yanchor='middle',
#                                   text='Count'.format(y_trace[11]),
#                                   font=dict(family='Arial',
#                                             size=16),
#                                   showarrow=False))
annotations.append(dict(xref='paper', yref='paper', x=0.75, y=0.95,
                              xanchor='left', yanchor='bottom',
                              text='Peak: 1.39M tickets',
                              font=dict(family='Arial',
                                        size=16,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
annotations.append(dict(xref='paper', yref='paper', x=-0.05, y=1.02,
                              xanchor='left', yanchor='bottom',
                              text='Ticket Count',
                              font=dict(family='Arial',
                                        size=16,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=-0.15, y=1.15,
                              xanchor='left', yanchor='bottom',
                              text='The Number of Car Tickers Issued by Month',
                              font=dict(family='Arial',
                                        size=22,
                                        color='black'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.12,
                              xanchor='center', yanchor='top',
                              text='Source: Kaggle',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations)

fig.show()

## Top 10 Ave/Str in 3 NYC Major County Roger_Ren

In [37]:
Q4 = rdd.map(lambda x: (((x[11]+' '+x[6]),x[11]),1))
Q4 = Q4.reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1],ascending=False)
Q4 = Q4.map(lambda x: (x[0][0],x[0][1],x[1])).filter(lambda x: x[1]!='')
Top10_street_count = sc.parallelize(Q4.take(200))
Top10_street_count_df = Top10_street_count.toDF()
Top10_street_count_df.createOrReplaceTempView("Top10_street_count_view")
Top10_street_count_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---+------+
|                _1| _2|    _3|
+------------------+---+------+
|       NY Broadway| NY|333613|
|        NY 3rd Ave| NY|259345|
|    NY Madison Ave| NY|196487|
|  NY Lexington Ave| NY|169317|
|        NY 2nd Ave| NY|152598|
|        NY 1st Ave| NY|151529|
|     Q Queens Blvd|  Q|131173|
|  NY Amsterdam Ave| NY|120289|
|        NY 5th Ave| NY|109150|
|        NY 8th Ave| NY|107327|
|         K 5th Ave|  K|107123|
|     Q Jamaica Ave|  Q|104717|
|        NY 6th Ave| NY| 99601|
|   NY Columbus Ave| NY| 90503|
|        NY 7th Ave| NY| 89088|
|        Q 37th Ave|  Q| 82235|
|K Coney Island Ave|  K| 78280|
|       NY Park Ave| NY| 76708|
|   Q Roosevelt Ave|  Q| 74486|
|BX White Plains Rd| BX| 72800|
+------------------+---+------+
only showing top 20 rows

In [38]:
%%sql -o Top10_street_count_table_NY
SELECT *
FROM Top10_street_count_view
where _2 = 'NY'
Order by _3 Desc
limit 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [39]:
%%sql -o Top10_street_count_table_K
SELECT _1,'Kings' as _2, _3
FROM Top10_street_count_view
where _2 = 'K'
Order by _3 Desc
limit 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [40]:
%%sql -o Top10_street_count_table_Q
SELECT _1,'Queens' as _2, _3
FROM Top10_street_count_view
where _2 = 'Q'
Order by _3 Desc
limit 10


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [41]:
%local

# add layout
layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)
fig = go.Figure(layout=layout)

# add 3 trace
fig.add_trace(go.Bar(
    y=Top10_street_count_table_NY['_3'],
    x=Top10_street_count_table_NY['_2'],
    name='NY',
    text = Top10_street_count_table_NY['_1'],
    marker=dict(
        color=sorted(px.colors.sequential.Plasma)
    )
))

fig.add_trace(go.Bar(
    y=Top10_street_count_table_K['_3'],
    x=Top10_street_count_table_K['_2'],
    name='Kings County',
    text = Top10_street_count_table_K['_1'],
    marker=dict(
        color=sorted(px.colors.sequential.Plasma)
    )
))

fig.add_trace(go.Bar(
    y=Top10_street_count_table_Q['_3'],
    x=Top10_street_count_table_Q['_2'],
    text = Top10_street_count_table_Q['_1'],
    name='Queen',
    marker=dict(
        color=sorted(px.colors.sequential.Plasma)
    )
))

fig.update_layout(barmode='stack')  # stack mode:['stack', 'group', 'overlay', 'relative']

fig.update_layout(
    title="Top 10 Ave/Str in 3 NYC Major County",
    xaxis_title="NYC County",
    yaxis_title="Number of Parking Tickets Issued",
    showlegend=False,
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="RebeccaPurple"
    )
)

fig.show()


# Date v.s Tickets _Xinyue Wang

In [42]:
month_count = rdd.map(lambda x: x[2]).map(lambda x: x[:3]+x[6:]).countByValue()
month = sorted(month_count.items(),key=lambda x: x[1],reverse=True)[:40]
month.pop(-7)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('PAS', 117)

In [43]:
month = sc.parallelize(month)
month_df = month.toDF(['Date','Ticket Count'])
month_df.createOrReplaceTempView("month_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
%%sql -o month_table
SELECT * FROM month_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [45]:
Q7 = rdd.map(lambda x: x[2]).countByValue()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
date_count_top20 = sorted(Q7.items(),key=lambda x: x[1],reverse=True)[:20]
date_count_top20 = sc.parallelize(date_count_top20)
date_count_top20_df = date_count_top20.toDF(['Date','Ticket Count'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
date_count_top20_df.createOrReplaceTempView("date_count_top20_df_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
%%sql -o date_count_top20_table
SELECT * FROM date_count_top20_df_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [49]:
%%local
x = date_count_top20_table['Date']
y = date_count_top20_table['Ticket Count']
fig = go.Figure(data=go.Bar(x=x,y=y),layout=go.Layout(title=go.layout.Title(text="Top20 Date that Get Tickets"),    
                                                      paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'))
fig.update_layout(xaxis={'type': 'category'})
fig.show()

# Ticket Counts Based On Time _Karsten Cao


In [50]:
%%local
import plotly
import plotly.graph_objects as go
import pandas as pd
import numpy as np

### Check County-Street Repeats
There are different counties in the NYC dataset. We would like to check if anything repeats. 

In [51]:
data.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[5] at RDD at PythonRDD.scala:53

In [52]:
split_rdd = data
columns = data.first()
def ci(col, dropped=None):
    return columns.index(col)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
# Get Counties
print((
    split_rdd
    .map(lambda x:(x[ci("Violation County")]))
    .distinct()
).collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['', 'BX', '0753A', '0938P', '0257P', 'KING', '0146P', '0150P', '1200P', 'MAN', 'NY', 'MH', 'RICH', '1030A', '0401P', '1221P', 'RC', 'Violation County', '0522P', 'QU', 'NEW Y', '0931A', '1208P', '0602P', 'Q', '0551P', 'NEWY', '103', '0217P', 'R', 'KINGS', '1203P', '0157P', 'NYC', 'K', 'QUEEN', 'BRONX', '1104A', '0901A']

In [54]:
for elem in (
    split_rdd
    .map(lambda x: (x[ci("Street Name")], x[ci("Violation County")]))
    .groupByKey()
).take(10): 
    print(elem[0], set(elem[1]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 {'', 'R', 'BX', '1200P', 'K', 'Q', 'NY'}
ROEBLING ST {'', 'NY', 'K'}
GRAND CONCOURSE {'', 'R', 'BX', 'K', 'Q', 'NY'}
N/W W 165 ST {'BX'}
121 ST {'', 'NY', 'Q'}
MASTERDAM AVE {'NY'}
SIDE OF 24 HUMBOLDT {'K'}
WEST 118 ST {'', 'NY'}
CLINTON AVE {'', 'R', 'BX', 'K', 'Q', 'NY'}
ORIENTAL BLVD {'K'}

### Takeaways
There are more than 5 counties. Some data has been inputted incorrectly. But the incorrect data should not affect the top aggregated results.

In [55]:
(
    split_rdd
    .map(lambda x: (x[ci("Issue Date")][:2], 1))
    .reduceByKey(lambda x, y: x + y)
    .filter(lambda x: x[0] in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
    .sortByKey()
).toDF().createOrReplaceTempView("month_count_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
%%sql -o month_df
SELECT * FROM month_count_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [57]:
%%local
x = month_df['_1'].replace({
                                1: "January",
                                2: "February",
                                3: "March",
                                4: "April",
                                5: "May",
                                6: "June",
                                7: "July",
                                8: "August",
                                9: "September",
                                10: "October",
                                11: "November",
                                12: "December",
                                })
y = month_df['_2']
fig = go.Figure(data=[go.Bar(x=x, y=y)])
fig.update_layout(title="Count of Issued Parking Tickets by Month")
fig.show()

### Info
For personal reference, the end of the fiscal year for the NYPD is June 30th. 

In [58]:
(
    split_rdd
    .filter(lambda x: x[ci("Issue Date")][:2] == "06")
    .map(lambda x: (violationinfo(x[ci("Violation Code")]), 1))
    .reduceByKey(lambda x, y: x + y)
    .sortBy(lambda x: x[1], ascending=False)
).toDF().createOrReplaceTempView("month6_viol_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
%%sql -o month6_v_df
SELECT * FROM month6_viol_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [60]:
%%local
x = month6_v_df['_1'][:10]
y = month6_v_df['_2'][:10]

fig = go.Figure(data=[go.Pie(labels=x,values=y,pull=[0.1, 0.1, 0, 0])],layout=go.Layout(title=go.layout.Title(text="Top10 Violation Reasons in NY County for June"), showlegend=False))
fig.update_layout(xaxis=dict(tickformat='digit'))
fig.show()

In [61]:
(
    split_rdd
    .filter(lambda x: x[ci("Issue Date")][:2] == "07")
    .map(lambda x: (violationinfo(x[ci("Violation Code")]), 1))
    .reduceByKey(lambda x, y: x + y)
    .sortBy(lambda x: x[1], ascending=False)
).toDF().createOrReplaceTempView("month7_viol_view")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
%%sql -o month7_v_df
SELECT * FROM month7_viol_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [63]:
%%local
x = month7_v_df['_1'][:10]
y = month7_v_df['_2'][:10]

fig = go.Figure(data=[go.Pie(labels=x,values=y,pull=[0.1, 0.1, 0, 0])],layout=go.Layout(title=go.layout.Title(text="Top10 Violation Reasons in NY County for July")))
fig.update_layout(xaxis=dict(tickformat='digit'))
fig.show()

In [64]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…